In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
#import necesseray libraries


In [18]:
# Define functions as per the instructions
def linear_function(x):
    return 3 * x + 1

def quadratic_function(x):
    return 5 * x**2 + 3 * x + 1

def cubic_function(x):
    return 7 * x**3 + 5 * x**2 + 3 * x + 1


In [ ]:
def normalize(data):
    return (data - data.mean()) / data.std()

def denormalize(data, original_mean, original_std):
    return data * original_std + original_mean

# Data generation function
def data_generation(function, batch_s=256):
    data = []
    x = 20 * np.random.randn(batch_s)  # Random inputs
    for i in range(batch_s):
        y = function(x[i])
        data.append([x[i], y])  # Append to the list

    data = torch.FloatTensor(data)
    x_min, x_max = data[:, 0].min(), data[:, 0].max()
    y_min, y_max = data[:, 1].min(), data[:, 1].max()
    data[:, 0] = normalize(data[:, 0])
    data[:, 1] = normalize(data[:, 1])
    return data, x_min, x_max, y_min, y_max

In [ ]:
# Generator class
class Generator(nn.Module):
    def __init__(self, latent_dim, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim + input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, z, x):
        combined = torch.cat((z, x), dim=1)
        return self.model(combined)

# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Training function
def train_gan(generator, discriminator, data_function, epochs=10000, batch_size=256, lr=0.0002):
    g_optimizer = optim.Adam(generator.parameters(), lr=lr)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
    criterion = nn.SmoothL1Loss()

    for epoch in range(epochs):
        # Generate real data
        real_data, x_min, x_max, y_min, y_max = data_generation(data_function, batch_size)
        x_values = real_data[:, 0].unsqueeze(1)

        # Generate fake data
        fake_noise = torch.randn(batch_size, 10)  # Latent dimension
        y_fake = generator(fake_noise, x_values)
        fake_data = torch.cat((x_values, y_fake), dim=1)

        # Labels for real and fake
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Train discriminator
        d_loss_real = criterion(discriminator(real_data), real_labels)
        d_loss_fake = criterion(discriminator(fake_data), fake_labels)
        d_loss = d_loss_real + d_loss_fake

        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train generator
        fake_noise = torch.randn(batch_size, 10)
        y_fake = generator(fake_noise, x_values)
        fake_data = torch.cat((x_values, y_fake), dim=1)
        g_loss = criterion(discriminator(fake_data), real_labels)

        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # Print losses
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


In [ ]:
def plot_results(generator, data_function, function_name):
    # Generate 2500 real samples
    real_data, x_min, x_max, y_min, y_max = data_generation(data_function, batch_s=2500)
    x_real = denormalize(real_data[:, 0], x_min, x_max)
    y_real = denormalize(real_data[:, 1], y_min, y_max)

    # Generate 2500 fake samples
    fake_noise = torch.randn(2500, 10)  # Latent dimension
    x_fake = torch.linspace(-1, 1, 2500).unsqueeze(1)
    combined_input = torch.cat((fake_noise, x_fake), dim=1)
    y_fake = generator(fake_noise, x_fake).detach()
    x_fake = denormalize(x_fake, x_min, x_max)
    y_fake = denormalize(y_fake, y_min, y_max)

    # Plot real and fake data
    plt.figure(figsize=(8, 6))
    plt.scatter(x_real.numpy(), y_real.numpy(), label="Original Data (2500)", alpha=0.6)
    plt.scatter(x_fake.numpy(), y_fake.numpy(), label="Generated Data (2500)", alpha=0.6)
    plt.title(function_name)
    plt.xlabel("Inputs")
    plt.ylabel("Outputs")
    plt.legend()
    plt.show()


    
# Parameters
noise_dim = 10
x_dim = 1
output_dim = 1

# Linear
print("Training GAN for Linear Data...")
linear_gen = Generator(noise_dim, x_dim, output_dim)
linear_disc = Discriminator(2)
train_gan(linear_gen, linear_disc, linear_function, epochs=10000)
plot_results(linear_gen, linear_function, "Linear Function: y = 3x + 1")

# Quadratic
print("Training GAN for Quadratic Data...")
quadratic_gen = Generator(noise_dim, x_dim, output_dim)
quadratic_disc = Discriminator(2)
train_gan(quadratic_gen, quadratic_disc, quadratic_function, epochs=10000)
plot_results(quadratic_gen, quadratic_function, "Quadratic Function: y = 5x^2 + 3x + 1")

# Cubic
print("Training GAN for Cubic Data...")
cubic_gen = Generator(noise_dim, x_dim, output_dim)
cubic_disc = Discriminator(2)
train_gan(cubic_gen, cubic_disc, cubic_function, epochs=15000)
plot_results(cubic_gen, cubic_function, "Cubic Function: y = 7x^3 + 5x^2 + 3x + 1")


TypeError: normalize() takes 1 positional argument but 3 were given